## Compare results for multiple results for 512 square datasets 
- Aug 11, 2020 : 512 square dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import subprocess as sp
import os
import glob
import sys

import itertools
import time

from ipywidgets import *

In [2]:
%matplotlib widget

In [3]:
sys.path.append('/global/u1/v/vpa/project/jpt_notebooks/Cosmology/Cosmo_GAN/repositories/lbann_cosmogan/3_analysis')
from modules_image_analysis import *

In [4]:
### Transformation functions for image pixel values
def f_transform(x):
    return 2.*x/(x + 4. + 1e-8) - 1.

def f_invtransform(s):
    return 4.*(1. + s)/(1. - s + 1e-8)

In [5]:
def f_compute_hist_spect(sample,bins):
    ''' Compute pixel intensity histograms and radial spectrum for 2D arrays
    Input : Image arrays and bins
    Output: dictionary with 5 arrays : Histogram values, errors and bin centers, Spectrum values and errors.
    '''
    ### Compute pixel histogram for row
    gen_hist,gen_err,hist_bins=f_batch_histogram(sample,bins=bins,norm=True,hist_range=None)
    ### Compute spectrum for row
    spec,spec_err=f_compute_spectrum(sample,plot=False)

    dict1={'hist_val':gen_hist,'hist_err':gen_err,'hist_bin_centers':hist_bins,'spec_val':spec,'spec_err':spec_err }
    return dict1


def f_compute_chisqr(dict_val,dict_sample):
    '''
    Compute chi-square values for sample w.r.t input images
    Input: 2 dictionaries with 4 keys for histogram and spectrum values and errors
    '''
    ### !!Both pixel histograms MUST have same bins and normalization!
    ### Compute chi-sqr
    ### Used in keras code : np.sum(np.divide(np.power(valhist - samphist, 2.0), valhist))
    ###  chi_sqr :: sum((Obs-Val)^2/(Val))
    
    chisqr_dict={}
    
    val_dr=dict_val['hist_val'].copy()
    val_dr[val_dr<=0.]=1.0    ### Avoiding division by zero for zero bins
    
    sq_diff=(dict_val['hist_val']-dict_sample['hist_val'])**2
    
    size=len(dict_val['hist_val'])
    l1,l2=int(size*0.3),int(size*0.7)
    keys=['chi_1a','chi_1b','chi_1c','chi_1']
    
    for (key,start,end) in zip(keys,[0,l1,l2,0],[l1,l2,None,None]):  # 4 lists : small, medium, large pixel values and full 
        chisqr_dict.update({key:np.sum(np.divide(sq_diff[start:end],val_dr[start:end]))})
    
    idx=None  # Choosing the number of histograms to use. Eg : -5 to skip last 5 bins
#     chisqr_dict.update({'chi_sqr1':})
    
    chisqr_dict.update({'chi_2':np.sum(np.divide(sq_diff[:idx],1.0))}) ## chi-sqr without denominator division
    chisqr_dict.update({'chi_imgvar':np.sum(dict_sample['hist_err'][:idx])/np.sum(dict_val['hist_err'][:idx])}) ## measures total spread in histograms wrt to input data
    
    idx=60
    spec_diff=(dict_val['spec_val']-dict_sample['spec_val'])**2
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec1':np.sum(spec_diff[:idx]/dict_sample['spec_val'][:idx]**2)})
    
    ### computing the spectral loss chi-square
    chisqr_dict.update({'chi_spec2':np.sum(spec_diff[:idx]/dict_sample['spec_err'][:idx]**2)})
    
    return chisqr_dict

In [6]:
bins=np.concatenate([np.array([-0.5]),np.arange(0.5,20.5,1),np.arange(20.5,100.5,5),np.arange(100.5,1000.5,50),np.array([2000])]) #bin edges to use
bins=f_transform(bins)   ### scale to (-1,1)

### Extract validation data
fname='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/norm_1_train_val.npy'
s_val=np.load(fname,mmap_mode='r')[5000:8000][:,0,:,:]
print(s_val.shape)

### Compute histogram and spectrum of raw data 
dict_val=f_compute_hist_spect(s_val,bins)
del(s_val)

(3000, 512, 512)


In [7]:
df_runs=pd.DataFrame([])

In [9]:
### Load raw images ( a different set of input images for comparison)
dict1={}
dict1.update({'label':'raw','fname':'/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/512_square/dataset1_smoothing_single_universe/train.npy'})
images=f_transform(np.load(dict1['fname'],mmap_mode='r')[:3000][:,:,:,0])
dict1['num_img']=images.shape[0]
### Compute spectrum and histograms
dict_sample=f_compute_hist_spect(images,bins)
### Compute chi squares
dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)

dict1.update(dict_sample)
dict1.update(dict_chisqrs)
del(images)
df_runs=df_runs.append(dict1,ignore_index=True)


In [14]:
### Load keras images
parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_from_other_code/exagan1/'
lst=['gen_imgs_1.npy','gen_imgs_2.npy']
for strg,fldr in zip([str(i) for i in range(len(lst))],lst):
    key='keras_'+strg
    fname=parent_dir+'run2_512_square_20epochs/models/'+fldr
    dict1={'label':key,'fname':fname}

    images=np.load(dict1['fname'])[:3000]     ### No transform needed here
#     print(np.max(images),np.min(images))

    dict1['num_img']=images.shape[0]
    ### Compute spectrum and histograms
    dict_sample=f_compute_hist_spect(images,bins)
    ### Compute chi squares
    dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
    dict1.update(dict_sample)
    dict1.update(dict_chisqrs)
    del(images)
    df_runs=df_runs.append(dict1,ignore_index=True)


In [18]:
# # ### Extract a few images generated by Lbann
# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/generate_images/'
# # fldr='20200806_065947_gen_img_exagan/'

# lst=['20200806_070111_gen_img_exagan/']
# names=['1_b64']

# for strg,fldr in zip(names,lst):
#     f_strg=parent_dir+fldr+'dump_outs/trainer0/model0/sgd.testing.epoch.*.step.*_gen_img_instance1_activation_output0.npy'
#     f_list=glob.glob(f_strg)
#     print(f_list)
#     key='lbann_'+strg
# #     fname=parent_dir+fldr+'/models/gen_imgs.npy'
#     dict1={'label':key,'fname':f_strg}
#     arr=[np.load(fname)[:,0,:,:] for fname in f_list]
#     images=np.vstack(arr)
#     dict1['num_img']=images.shape[0]
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

# # print(s_lbann.shape,np.max(s_lbann))


In [19]:
# ### Extract some LBANN training run data

# parent_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/results_data/20200805_124242_batchsize_64/dump_outs/trainer0/model0/'
# lst=[(5,16920),(6,20200),(7,23680),(4,14720)]

# for count,run in enumerate(lst):
#     epoch,step=run[0],run[1]
#     key='lbann_train_{0}-{1}'.format(epoch,step)
#     fname=parent_dir+'sgd.training.epoch.{0}.step.{1}_gen_img_instance1_activation_output0.npy'.format(epoch,step)
#     dict1={'label':key,'fname':fname}
#     images=np.load(fname)[:,0,:,:]
#     dict1['num_img']=images.shape[0]
#     print(dict1)
#     ### Compute spectrum and histograms
#     dict_sample=f_compute_hist_spect(images,bins)
#     ### Compute chi squares
#     dict_chisqrs=f_compute_chisqr(dict_val,dict_sample)
    
#     dict1.update(dict_sample)
#     dict1.update(dict_chisqrs)
#     del(images)
#     df_runs=df_runs.append(dict1,ignore_index=True)

# # print(s_lbann.shape,np.max(s_lbann))



In [20]:
df_runs.columns

Index(['label', 'fname', 'hist_bin_centers', 'hist_err', 'hist_val',
       'spec_err', 'spec_val', 'chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img'],
      dtype='object')

In [21]:
col_list=['label','fname', 'hist_bin_centers', 'hist_err', 'hist_val','spec_err', 'spec_val','chi_1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2',
       'chi_imgvar', 'chi_spec1', 'chi_spec2', 'num_img']
df_runs=df_runs.reindex(columns=col_list)

In [22]:
df_runs

,label,fname,hist_bin_centers,hist_err,hist_val,spec_err,spec_val,chi_1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec1,chi_spec2,num_img
0,raw,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0003194802447942272, 0.00023159762184637347...","[1.1192594268581553, 0.9588884025085721, 0.258...","[6611332.122177465, 20227.873858650168, 11845....","[51085258635.40556, 2589831.4952690783, 229847...",0.000076,0.000004,0.000009,0.000063,8.515733e-07,1.022982,0.000841,124.958102,3000.0
1,keras_0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0005839778961293528, 0.0002965655136510553,...","[1.3650680016291195, 0.6150514594848948, 0.207...","[15529065.579627804, 41986.235038721, 26682.71...","[54199012378.350876, 3866557.133600076, 318561...",0.268257,0.190081,0.004148,0.074028,1.810116e-01,2.146930,1.541081,118362.263513,3000.0
2,keras_1,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...,"[-1.031746031584782, -0.6161616169043975, -0.3...","[0.0009429652883415037, 0.0007760079159700334,...","[1.314512303196623, 0.6849693966301789, 0.2230...","[18271015.131826952, 18432.181935205295, 11269...","[53505418547.271355, 1834246.9643073464, 16543...",0.155952,0.117544,0.017693,0.020714,1.142549e-01,2.001993,0.878018,76776.947480,3000.0


### Compare chi-squares

In [17]:
chi_keys=['label','chi_1','chi_spec1', 'chi_1a', 'chi_1b', 'chi_1c', 'chi_2','chi_imgvar', 'chi_spec2', 'num_img','fname']
df_runs[chi_keys]

,label,chi_1,chi_spec1,chi_1a,chi_1b,chi_1c,chi_2,chi_imgvar,chi_spec2,num_img,fname
0,raw,0.000076,0.000841,0.000004,0.000009,0.000063,8.515733e-07,1.022982,124.958102,3000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
1,keras_0,0.268257,1.541081,0.190081,0.004148,0.074028,1.810116e-01,2.146930,118362.263513,3000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...
2,keras_1,0.155952,0.878018,0.117544,0.017693,0.020714,1.142549e-01,2.001993,76776.947480,3000.0,/global/cfs/cdirs/m3363/vayyar/cosmogan_data/r...


In [28]:

def f_plot_hist_spec_best(df,dict_bkg):

    fig=plt.figure(figsize=(6,6))
    ax1=fig.add_subplot(121)
    ax2=fig.add_subplot(122)
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):

        x1=row.hist_bin_centers
        y1=row.hist_val
        yerr1=row.hist_err
        x1=f_invtransform(x1)

        y2=row.spec_val
        yerr2=row.spec_err
        x2=np.arange(len(y2))

        label=row.label
        ax1.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
    #     ax2.errorbar(x2,y2,yerr2,marker=marker,markersize=5,linestyle='',label='{0}-{1}'.format(epoch,step))

        ax2.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
        ax2.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
    x=f_invtransform(x)
    ax1.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
    ax1.set_title('pixel intensity')
    y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
    x=np.arange(len(y))
    ax2.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
    ax2.set_title('Spectrum')


    plt.legend()
    # plt.yscale('log')
    ax1.set_xscale('symlog',linthreshx=50)
    ax1.set_yscale('log')
    ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:

def f_plot_hist_spec_best(df,dict_bkg,plot_type):

    fig=plt.figure(figsize=(6,6))
    for (i,row),marker in zip(df.iterrows(),itertools.cycle('>^*sDHPdpx_')):
        
        label=row.label
        if plot_type=='hist':
            x1=row.hist_bin_centers
            y1=row.hist_val
            yerr1=row.hist_err
            x1=f_invtransform(x1)
            
            plt.errorbar(x1,y1,yerr1,marker=marker,markersize=5,linestyle='',label=label)
        if plot_type=='spec':
            
            y2=row.spec_val
            yerr2=row.spec_err
            x2=np.arange(len(y2))
            
            plt.fill_between(x2, y2 - yerr2, y2 + yerr2, alpha=0.4)
            plt.plot(x2, y2, marker=marker, linestyle=':',label=label)

    ### Plot input data
    if plot_type=='hist':
        x,y,yerr=dict_bkg['hist_bin_centers'],dict_bkg['hist_val'],dict_bkg['hist_err']
        x=f_invtransform(x)
        plt.errorbar(x, y,yerr,color='k',linestyle='-',label='bkgnd')   
        plt.title('Pixel Intensity Histogram')
    if plot_type=='spec':
        y,yerr=dict_bkg['spec_val'],dict_bkg['spec_err']
        x=np.arange(len(y))
        plt.fill_between(x, y - yerr, y + yerr, color='k',alpha=0.8)
        plt.title("Spectrum")
    plt.legend(bbox_to_anchor=(0.3, 0.75),ncol=2, fancybox=True, shadow=True,prop={'size':6})
    plt.yscale('log')
#     ax1.set_xscale('symlog',linthreshx=50)
#     ax1.set_yscale('log')
#     ax2.set_yscale('log')


f_plot_hist_spec_best(df_runs,dict_val,'hist')
f_plot_hist_spec_best(df_runs,dict_val,'spec')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
def f_widget_compare(sample_names,sample_dict,Fig_type='pixel',rescale=True,log_scale=True,bins=25,mode='avg',normalize=True,bkgnd=[]):
    '''
    Module to make widget plots for pixel intensity or spectrum comparison for multiple sample sets
    '''
    
    ### Crop out large pixel values
#     for key in sample_names:
#         print(sample_dict[key].shape)
#         sample_dict[key]=np.array([arr for arr in sample_dict[key] if np.max(arr)<=0.994])
#         print(sample_dict[key].shape)
    
    img_list=[sample_dict[key] for key in sample_names]
    label_list=list(sample_names)
    
    if rescale: 
        for count,img in enumerate(img_list):
            img_list[count]=f_invtransform(img)
        if len(bkgnd): bkgnd=f_invtransform(bkgnd)
#         hist_range=(0,2000)
    else:
        bins=f_transform(bins)
#         hist_range=(-1,0.996)
    assert Fig_type in ['pixel','spectrum'],"Invalid mode %s"%(mode)
    
    if Fig_type=='pixel':
#         f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=hist_range)
        f_compare_pixel_intensity(img_lst=img_list,label_lst=label_list,normalize=normalize,log_scale=log_scale, mode=mode,bins=bins,hist_range=None,bkgnd_arr=bkgnd)

    elif Fig_type=='spectrum':
        f_compare_spectrum(img_lst=img_list,label_lst=label_list,log_scale=log_scale,bkgnd_arr=bkgnd)



In [ ]:
# dict_samples={'raw': s_raw,'keras1':s_keras[0],'keras2':s_keras[1],'s_new1':s_new1,'s_new2':s_new2,'s_new3':s_new3}

bkgnd=s_raw
# bkgnd=[]
interact_manual(f_widget_compare,sample_dict=fixed(dict_samples),
                sample_names=SelectMultiple(options=dict_samples.keys()),
                Fig_type=ToggleButtons(options=['pixel','spectrum']),
                bins=fixed(bins),
                mode=['avg','simple'],bkgnd=fixed(bkgnd))